In [1]:
import tensorflow as tf
#from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Lambda,Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l1,l2,l1_l2
from tensorflow.keras.models import Sequential  #用來啟動 NN
from tensorflow.keras.layers import Conv2D  # Convolution Operation
from tensorflow.keras.layers import MaxPooling2D # Pooling
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense # Fully Connected Networks
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import concatenate, AveragePooling2D, UpSampling2D, add, Multiply, GlobalAveragePooling2D
import tensorflow.keras as keras  
from tensorflow.keras import backend as K
import numpy as np
import pandas as pd
import os
import random
import re
import time
from random import shuffle
from matplotlib import pyplot as plt
from scipy.stats import norm
import statistics

In [2]:
def data_collect(Path):
    files=os.listdir(Path)
    random.shuffle(files)
    Train_Lstm=[]
    Label=[]
    Train=[]
    Train
    count=1
    Train_Pdf=[]
    Time_lists = []
    for file in files:
        Time_lists.append(file)
        start=time.time()
        Break=0
        global Train_pdf, Label_Pdf
        Train_lstm=[]
        file_data=os.listdir(Path+file)
        
        ##CNN_PDF
        #CSFV2
        df_csfv2 = pd.read_csv(Path+file+'/'+file_data[0],header=None)
        df_era5 = pd.read_csv(Path+file+'/'+file_data[1],header=None)
        for column in df_csfv2.columns[2:12]:
            if column == df_csfv2.columns[2]:
                Train_pdf=np.array(np.array(df_csfv2[column]).reshape(61,111,1))
            else:
                Train_pdf=np.append(Train_pdf,np.array(df_csfv2[column]).reshape(61,111,1),axis=2)
                
        #ERAF
        for column in df_era5.columns[2:7]:
            Train_pdf=np.append(Train_pdf,np.array(df_era5[column]).reshape(61,111,1),axis=2)

        ##LSTM
        #ERA5
        df_1 = pd.read_csv(Path+file+'/'+file_data[3])
        for column in df_1.columns[1:91]:
            if' NaN' in list(df_1[column])[:]:
                Break=1
                break
            Train_lstm.append(list(df_1[column]))
            
        #CSFV2
        df_2 = pd.read_csv(Path+file+'/'+file_data[2])
        for column in df_2.columns[1:181]:
            if' NaN' in list(df_2[column])[:]:
                Break=1
                break
            Train_lstm.append(list(df_2[column]))
        if Break==1:
            continue

        ##LABEL_PDF 
        for column in df_era5.columns[7:17]:
            if column == df_era5.columns[7]:
                Label_Pdf=np.array(np.array(df_era5[column]).reshape(61,111,1))
                #print(Label_Pdf)
            else:
                Label_Pdf=np.append(Label_Pdf,np.array(df_era5[column]).reshape(61,111,1),axis=2)

        Train_Pdf.append(Train_pdf)
        Train_Lstm.append(Train_lstm)
        Label.append(Label_Pdf)
        end=time.time()
        Time=(end-start)*(len(files)-count)
        print('%d / %d , Time : %d : %d : %d'%(count,len(files),int(Time/3600),int(Time%3600/60),Time%3600%60),end='\r')
        count=count+1
    Train.append(Train_Pdf)
    Train.append(Train_Lstm)
    Label=np.array(Label)
    return Train, Label, Time_lists

In [ ]:
#資料位置
Path='D:/weather/data_180/Test/'
#Path='D:/weather/data_180/Train/'

#載入資料
Valid,Valid_Label,Time_lists=data_collect(Path)

#模型位置
model_path = 'D:/weather/model/LSMT_Restnet_new_data_my_loss_only_LSTM_180days.h5'

#預測CSV儲存位置
Path_perdict_csv = 'D:/weather/Predict_csv/LSTM_180/'

In [ ]:
Valid_Pdf=Valid[0]
Valid_Lstm=Valid[1]
Valid_Pdf=np.array(Valid_Pdf)
Valid_Lstm=np.array(Valid_Lstm)
print('Valid_Lstm:',Valid_Lstm.shape)
print('Train_Pdf:',Valid_Pdf.shape)

In [ ]:
def mcustom_loss(y_true, y_pred):

    trueAvg = (y_true[:,:,:,0][:,:,:,np.newaxis]-Mean[0])/(Max[0]-Min[0])
    trueStd = (y_true[:,:,:,1][:,:,:,np.newaxis]-Mean[1])/(Max[1]-Min[1])
    trueMax = (y_true[:,:,:,2][:,:,:,np.newaxis]-Mean[2])/(Max[2]-Min[2])
    trueMin = (y_true[:,:,:,3][:,:,:,np.newaxis]-Mean[3])/(Max[3]-Min[3])
    trueSkw = (y_true[:,:,:,4][:,:,:,np.newaxis]-Mean[4])/(Max[4]-Min[4])

    predAvg = (y_pred[:,:,:,0][:,:,:,np.newaxis]-Mean[0])/(Max[0]-Min[0])
    predStd = (y_pred[:,:,:,1][:,:,:,np.newaxis]-Mean[1])/(Max[1]-Min[1])
    predMax = (y_pred[:,:,:,2][:,:,:,np.newaxis]-Mean[2])/(Max[2]-Min[2])
    predMin = (y_pred[:,:,:,3][:,:,:,np.newaxis]-Mean[3])/(Max[3]-Min[3])
    predSkw = (y_pred[:,:,:,4][:,:,:,np.newaxis]-Mean[4])/(Max[4]-Min[4])
    
    trueAvg_180 = (y_true[:,:,:,5][:,:,:,np.newaxis]-Mean[5])/(Max[5]-Min[5])
    trueStd_180 = (y_true[:,:,:,6][:,:,:,np.newaxis]-Mean[6])/(Max[6]-Min[6])
    trueMax_180 = (y_true[:,:,:,7][:,:,:,np.newaxis]-Mean[7])/(Max[7]-Min[7])
    trueMin_180 = (y_true[:,:,:,8][:,:,:,np.newaxis]-Mean[8])/(Max[8]-Min[8])
    trueSkw_180 = (y_true[:,:,:,9][:,:,:,np.newaxis]-Mean[9])/(Max[9]-Min[9])

    predAvg_180 = (y_pred[:,:,:,5][:,:,:,np.newaxis]-Mean[5])/(Max[5]-Min[5])
    predStd_180 = (y_pred[:,:,:,6][:,:,:,np.newaxis]-Mean[6])/(Max[6]-Min[6])
    predMax_180 = (y_pred[:,:,:,7][:,:,:,np.newaxis]-Mean[7])/(Max[7]-Min[7])
    predMin_180 = (y_pred[:,:,:,8][:,:,:,np.newaxis]-Mean[8])/(Max[8]-Min[8])
    predSkw_180 = (y_pred[:,:,:,9][:,:,:,np.newaxis]-Mean[9])/(Max[9]-Min[9])

    loss = ((K.square(predMin - trueMin) + K.square(predMax - trueMax) + K.square(predSkw - trueSkw) +
            K.square(predAvg - trueAvg) + K.square(predStd - trueStd))+
            (K.square(predMin_180 - trueMin_180) + K.square(predMax_180 - trueMax_180) + K.square(predSkw_180 - trueSkw_180) +
            K.square(predAvg_180 - trueAvg_180) + K.square(predStd_180 - trueStd_180)))/10
    loss = K.mean(loss, axis=-1)

    return loss

In [ ]:
Max=[307.9367, 17.7063, 313.7215, 304.4253, 3.8382,310.1568,18.0352,314.7128,306.1226,2.7172]
Min=[232.6118, 0.1771, 244.0102, 204.4635, -4.7362,234.9248,0.1928,248.2585,220.1559,-4.1973]
Mean=[287.3242734287198,
     3.349621542723795,
     293.65658980295865,
     279.6666574399456,
     -0.25665387472126,
     288.3584796428863,
     3.2974777666300223,
     294.7044402373835,
     281.08416123969715,
     -0.15500007492913936]

In [ ]:
from tensorflow import keras
model = load_model(model_path,custom_objects={'mcustom_loss': mcustom_loss})

In [ ]:
batchSize=1
y_predict=model.predict(Valid_Lstm,batchSize)

In [ ]:
for day in range(len(y_predict)):
    CSV = []
    for i in range(61):
        for j in range(111):
            CSV_PER = []
            CSV_PER.append(i+1)
            CSV_PER.append(j+1)
            for k in y_predict[day][i][j]:
                CSV_PER.append(k)
            CSV.append(CSV_PER)
    df= pd.DataFrame(CSV,columns=['X','Y','Mean_90','STD_90','Maximum_90','Minimum_90','Skewness_90','Mean_180','STD_180','Maximum_180','Minimum_180','Skewness_180'])
    df.to_csv(Path_perdict_csv+Time_lists[day]+'.csv',index=False)

In [ ]:
def PDF_point(i, j, data, start, end, Pic, DATE):
    PDF=[0,0,0,0,0,0,0,0,0,0]
    PDF_csfv2=[0,0,0,0,0,0,0,0,0,0]
    LABEL = ['Mean_90','STD_90','Maximum_90','Minimum_90','Skewness_90','Mean_180','STD_180','Maximum_180','Minimum_180','Skewness_180']
    Era5=[]
    Csfv2=[]
    Predict=[]
    print('第'+str(Pic+1)+'天:',DATE)
    print("Point:",i+1,j+1)
    DATA = []
    for pdf in range(start, end):                
        count=Valid_Label[Pic][i][j][pdf]
        count_data=data[i][j][pdf]
        count_csfv2=Valid_Pdf[Pic][i][j][5+pdf]
        Era5.append(round(count,2))
        Csfv2.append(round(count_csfv2,2))
        Predict.append(round(count_data,2))
        #print(Pic+1,pdf,'ERA5:',round(count,2),"CSFV2:",round(count_csfv2,2),'Predict:',round(count_data,2))
        DATA.append([LABEL[pdf],round(count,2),round(count_csfv2,2),round(count_data,2),round(count,2)-round(count_csfv2,2),round(count,2)-round(count_data,2)])
    df = pd.DataFrame(DATA,columns=['Type','ERA5','CSFV2','Predict','Ori_diff','Pre_deff'])
    print(df)
    Era5_pdf = np.arange(Era5[3],Era5[2],0.1)
    Csfv2_pdf = np.arange(Csfv2[3],Csfv2[2],0.1)
    Predict_pdf = np.arange(Predict[3],Predict[2],0.1)
    plt.plot(Era5_pdf, norm.pdf(Era5_pdf, Era5[0], Era5[1]),label='ERA5' )
    plt.plot(Csfv2_pdf, norm.pdf(Csfv2_pdf, Csfv2[0], Csfv2[1]),label='CSFV2' )
    plt.plot(Predict_pdf, norm.pdf(Predict_pdf, Predict[0], Predict[1]),label='Predict' )
    plt.legend()
    plt.show()    
    plt.show()    

In [ ]:
def PDF_average(data, start, end, Pic, DATE):
    PDF=[0,0,0,0,0,0,0,0,0,0]
    PDF_csfv2=[0,0,0,0,0,0,0,0,0,0]
    LABEL = ['Mean_90','STD_90','Maximum_90','Minimum_90','Skewness_90','Mean_180','STD_180','Maximum_180','Minimum_180','Skewness_180']
    Era5=[]
    Csfv2=[]
    Predict=[]
    print('第'+str(Pic+1)+'天:',DATE)
    DATA = []
    for pdf in range(start,end):
        count=0
        count_data=0
        count_csfv2=0
        for i in range(len(data)):
            for j in range(len(data[0])):
                count=count+Valid_Label[Pic][i][j][pdf]
                count_data=count_data+data[i][j][pdf]
                count_csfv2=count_csfv2+Valid_Pdf[Pic][i][j][5+pdf]
        #print(LABEL[pdf],'ERA5:',round(count/(61*111),2),"CSFV2:",round(count_csfv2/(61*111),2),'Predict:',round(count_data/(61*111),2))
        DATA.append([LABEL[pdf],round(count/(61*111),2),round(count_csfv2/(61*111),2),round(count_data/(61*111),2),round(count/(61*111),2)-round(count_csfv2/(61*111),2),round(count/(61*111),2)-round(count_data/(61*111),2)])
        Era5.append(round(count/(61*111),2))
        Csfv2.append(round(count_csfv2/(61*111),2))
        Predict.append(round(count_data/(61*111),2))
    df = pd.DataFrame(DATA,columns=['Type','ERA5','CSFV2','Predict','Ori_diff','Pre_deff'])
    print(df)
    Era5_pdf = np.arange(Era5[3],Era5[2],0.1)
    Csfv2_pdf = np.arange(Csfv2[3],Csfv2[2],0.1)
    Predict_pdf = np.arange(Predict[3],Predict[2],0.1)
    plt.plot(Era5_pdf, norm.pdf(Era5_pdf, Era5[0], Era5[1]),label='ERA5' )
    plt.plot(Csfv2_pdf, norm.pdf(Csfv2_pdf, Csfv2[0], Csfv2[1]),label='CSFV2' )
    plt.plot(Predict_pdf, norm.pdf(Predict_pdf, Predict[0], Predict[1]),label='Predict' )
    plt.legend()
    plt.show()

In [ ]:
#單點作圖PDF
Pic=0
for data in y_predict:
    for i in range(len(data)):
        for j in range(len(data[0])):
            
            #前90天
            print('前90天')
            PDF_point(i, j, data, 0, 5, Pic)
            
            #後90天
            print('後90天')
            PDF_point(i, j, data, 5, 10, Pic)
            
        Pic=Pic+1

In [ ]:
#平均作圖PDF
Pic=0
for data in y_predict:
    
    #前90天
    print('前90天')
    PDF_average(data, 0, 5, Pic)
    
    #後90天
    print('後90天')
    PDF_average(data, 5, 10, Pic)
    
    Pic=Pic+1